### Check epics connection

In [1]:
from epics import caget

In [2]:
# Make sure these lines are included in your bash_rc or bash_profile.
#######
# alias epicsproxy="ssh -fN -L <some number over 24000>:lcls-prod01.slac.stanford.edu:5068 slac"
# export EPICS_CA_NAME_SERVERS=localhost:<same number over 24000>
# alias epicstest="caget KLYS:LI22:11:KPHR"
#######

## In a terminal, run "epicsproxy" in and enter SLAC credential to establish connection.
## Then run "epicstest". If the connection worked, a number should be returned. 
## If a number is returned, yet this cell fails, re-start jupyter. 

#caget("QUAD:HTR:440:BACT")
caget("OTRS:DIAG0:525:XRMS")

cannot connect to OTRS:DIAG0:525:XRMS


## Choose and slice Bmad lattice

In [3]:
from bmad_live import get_DM_GOOD, get_pvdata, tao_commands, set_htr_twiss, run1

In [5]:
MODEL = 'sc_bsyd' # or sc_diag0 or sc_inj

BEGELE = 'BEGINNING'

#ENDELE = 'OTR0H04'
ENDELE = "ENDCOL0"

## Do not include sections beyond the beam's reach!!!
#ENDELE = 'END'
#ENDELE = 'ENDL3B'

In [6]:
from pytao import Tao

tao = Tao(f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init  -slice  {BEGELE}:{ENDELE}')

# Master switches for element scaling
tao.cmd('set ele quad::* field_master = T')
#tao.cmd('set ele lcavity::CAVL* autoscale_amplitude = T') # field_au
if MODEL == 'sc_inj':
    tao.cmd('set ele lcavity::CAVL* autoscale_phase = T')

In [7]:
MONITOR, DM_GOOD = get_DM_GOOD(tao, MODEL)

Bad PV: BPMS:COL0:940:Y
Bad PV: BPMS:COL0:480:X
Bad PV: BPMS:COL0:720:TMIT
Bad PV: BPMS:COL0:880:Y
Bad PV: SOLN:GUNB:823:BACT
Bad PV: BPMS:HTR:460:Y
Bad PV: BPMS:COL0:400:TMIT
Bad PV: BPMS:HTR:980:X
Bad PV: QUAD:COL0:260:BACT
Bad PV: BPMS:COL0:560:Y
Bad PV: BPMS:COL0:240:X
Bad PV: BPMS:COL0:135:X
Bad PV: QUAD:HTR:860:BACT
Bad PV: ACCL:L0B:0170:PACTMEAN
Bad PV: BPMS:COL0:720:X
Bad PV: BPMS:HTR:120:Y
Bad PV: QUAD:COL0:480:BACT
Bad PV: QUAD:HTR:440:BACT
Bad PV: QUAD:COL0:720:BACT
Bad PV: ACCL:L0B:0160:AACTMEAN
Bad PV: QUAD:GUNB:212:1:BACT
Bad PV: BPMS:COL0:560:X
Bad PV: BPMS:COL0:280:X
Bad PV: BPMS:HTR:120:X
Bad PV: BPMS:COL0:560:TMIT
Bad PV: BPMS:HTR:460:TMIT
Bad PV: BPMS:COL0:800:X
Bad PV: BPMS:COL0:320:Y
Bad PV: BPMS:HTR:540:Y
Bad PV: QUAD:L0B:0185:BACT
Bad PV: ACCL:GUNB:455:AACT_AVG
Bad PV: BPMS:HTR:365:TMIT
Bad PV: BPMS:HTR:860:TMIT
Bad PV: QUAD:HTR:120:BACT
Bad PV: QUAD:COL0:800:BACT
Bad PV: BPMS:HTR:120:TMIT
Bad PV: BPMS:COL0:135:TMIT
Bad PV: BPMS:COL0:240:Y
Bad PV: ACCL:L0B:0180:A

In [16]:
#%%tao
tao.cmd("set ele CAVL011 is_on = T")

[]

CAC: error = "Network is unreachable" sending UDP msg to 192.168.1.255:5064


In [9]:
# Scale Tao plots using Tao magic

#%%tao
#place top energy
#place middle orbit
#x_s top 0 100
#sc *
#tao.cmd('set global plot_on = F')

In [10]:
# Updating tao live-model once

run1(tao, MONITOR, DM_GOOD)

dt: 0.1279299259185791


CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"


In [10]:
set_htr_twiss(tao)

In [11]:
# mat2 x, y for PyEmittance
def get_mats():
    mat6 = tao.matrix('Q0H01#2', 'OTR0H04')['mat6']
    
    
    #mat6 = tao.matrix('QDG004#2', 'OTRDG02')['mat6']
    
    mat2x = mat6[0:2, 0:2]
    mat2y = mat6[2:4, 2:4]
    return mat2x, mat2y

In [12]:
get_mats()

(array([[-3.12456865,  4.01328803],
        [ 1.69661397, -2.49922515]]),
 array([[3.51169476, 5.37844322],
        [1.3110681 , 2.29276912]]))

# Optics compensation

In [72]:
# Chris cheating

def set_OTR0H04():
    cmds = f"""
set dat OTR0H04.midtwiss[1]|meas = 5
set dat OTR0H04.midtwiss[2]|meas = 0
set dat OTR0H04.midtwiss[3]|meas = 5
set dat OTR0H04.midtwiss[4]|meas = 0
vv
vd
use dat OTR0H04.midtwiss[1:4]
use var begtwiss[1:4]
run 
run
""".split('\n')
    for cmd in cmds:
        tao.cmd(cmd)
    
set_OTR0H04()

In [71]:
%%tao
SLD

-------------------------
Tao> SLD
-------------------------
Tao> 


In [62]:
# Updating tao live-model once

run1(tao, MONITOR, DM_GOOD)

dt: 0.1461658477783203


In [63]:
target_twiss_at_OTRC006 = tao.ele_twiss('OTRC006')
target_twiss_at_OTRC006;

In [64]:
### Set the target Twiss to the current ( undistorted ) Twiss
target_twiss_at_ENDHTR = tao.ele_twiss('ENDHTR')
target_twiss_at_ENDHTR

{'mode_flip': False,
 'beta_a': 53797.6848388652,
 'alpha_a': 8389.31709154993,
 'gamma_a': 1308.24667405997,
 'phi_a': 9.42649184653333,
 'eta_a': -3.12972839527373e-17,
 'etap_a': 5.80137611377415e-18,
 'beta_b': 2407.83276825869,
 'alpha_b': -887.473212091431,
 'gamma_b': 327.103157894753,
 'phi_b': 15.7046349909977,
 'eta_b': 8.21152593010106e-17,
 'etap_b': 2.31425331521468e-17,
 'eta_x': -5.14403433007268e-17,
 'etap_x': 9.5351653951773e-18,
 'eta_y': 0.0,
 'etap_y': 0.0}

In [13]:
#target_twiss_manual = {'beta_a':5.012, 'alpha_a':0.049,'beta_b':5.012,'alpha_b':0.049,'eta_x':0.0,'eta_y':0.0}

In [57]:
def optimize_twiss_cmds(dat_name, var_name, desired_twiss):
    """
        dat_name: STRING
        var_name: STRING
        desired_twiss : dictionary from tao.ele_twiss("ele_name")
    """
    
    out = f""" 
set global lattice_calc_on = F
!set global optimizer = lmdif 
set global optimizer = lm

veto dat *
veto var *

set dat {dat_name}[1]|meas = {desired_twiss['beta_a']}
set dat {dat_name}[2]|meas = {desired_twiss['alpha_a']}
set dat {dat_name}[3]|meas = {desired_twiss['beta_b']}
set dat {dat_name}[4]|meas = {desired_twiss['alpha_b']}
!set dat {dat_name}[5]|meas = {desired_twiss['eta_x']}
!set dat {dat_name}[6]|meas = {desired_twiss['etap_x']}

use dat {dat_name}[1:4]
use var {var_name}

set global lattice_calc_on = T

!show top

run
run
run
run
run
run

!show dat {dat_name}
!show merit
"""
    return out.split('\n')

In [17]:
TWISS_CMDS = optimize_twiss_cmds("HTR.endtwiss", "q.HTR[5:9]", target_twiss_at_ENDHTR )
tao.cmds(TWISS_CMDS);

In [18]:
tao.cmd('SLD')
print('Merit:', tao.merit()[0])
tao.cmd('change ele Q0H01 k1 @-0.1')
print('Merit:', tao.merit()[0])
tao.cmd('SLD')
print('Merit:', tao.merit()[0])

Merit:   1.66771264439763E+05
Merit:   7.50712841088190E+08
Merit:   1.66771264439763E+05


In [54]:
rematching_quads = 'q.HTR[5:9]'

var_info = tao.var_v_array(rematching_quads)
N_var = len(var_info)
q_names = [var_info[i]['var_attrib_name'][0:5] for i in range(N_var)]
q_names

dict_b1_grad = {q_names[i]:tao.ele_gen_attribs(q_names[i])['B1_GRADIENT'] for i in range(N_var)}

In [52]:
# Alternative way
#q_names = [q_dat[i]['var_attrib_name'] for i in range(8)]
#{q_names[i]:tao.lat_list(q_names[i], 'ele.b1_gradient', flags='-no_slaves')[0] for i in range(8)}

In [95]:
def twiss_optimizer(quad_name, k1_strength, dat_name, var_name, desired_twiss, reset_lat=False, debug=False):
    
    if debug:
        print(f'Rematching optics for {quad_name} with k1 = {k1_strength} 1/m^2')
    if reset_lat:
        tao.cmd('SLD')

    # Perturb
    tao.cmd(f'change ele {quad_name} k1 @{k1_strength}')
       
    TWISS_CMDS = optimize_twiss_cmds(dat_name, var_name, desired_twiss)
    
    merit = float(tao.merit()[0])
    print(f"Old marit: {merit}")   
    
    
    tao.cmds(TWISS_CMDS);
    
    
    merit = float(tao.merit()[0]) 
    assert merit < 1E-16, f"merit did NOT drop low enough: {merit}!!"
    print(f"New merit: {merit}")
        
        
    
    var_info = tao.var_v_array(var_name)
    N_var = len(var_info)
    q_names = [var_info[i]['var_attrib_name'][0:5] for i in range(N_var)]
 

    dict_b1_grad = {q_names[i]:tao.ele_gen_attribs(q_names[i])['B1_GRADIENT'] for i in range(N_var)}

    return dict_b1_grad
    

In [96]:
def twiss_optimizer2(quad_name, k1_strength, dat_name, var_name, desired_twiss, reset_lat=False, debug=False):
    
    #if debug:
    #    print(f'Rematching optics for {quad_name} with k1 = {k1_strength} 1/m^2')
    #if reset_lat:
    #    tao.cmd('SLD')

    # Perturb
    #tao.cmd(f'change ele {quad_name} k1 @{k1_strength}')
       
    TWISS_CMDS = optimize_twiss_cmds(dat_name, var_name, desired_twiss)
    
    merit = float(tao.merit()[0])
    print(f"Old merit: {merit}")   
    
    
    tao.cmds(TWISS_CMDS);
    
    
    merit = float(tao.merit()[0]) 
    assert merit < 1E-16, f"merit did NOT drop low enough: {merit}!!"
    print(f"New merit: {merit}")
        
        
    
    var_info = tao.var_v_array(var_name)
    N_var = len(var_info)
    q_names = [var_info[i]['var_attrib_name'][0:5] for i in range(N_var)]
 

    dict_b1_grad = {q_names[i]:tao.ele_gen_attribs(q_names[i])['B1_GRADIENT'] for i in range(N_var)}

    return dict_b1_grad
    

In [65]:
# Updating tao live-model once

run1(tao, MONITOR, DM_GOOD)

dt: 0.12140607833862305


In [99]:
caget('QUAD:HTR:120:BACT')

-2.20040419577489

In [102]:
1.7687386 * 1.2440000E-01 * -10

-2.2003108184

CA.Client.Exception...............................................
    Context: "localhost:24111"
    Source File: ../tcpiiu.cpp line 926
    Current Time: Tue Nov 22 2022 11:45:07.805106993
..................................................................
CA.Client.Exception...............................................
    Context: "localhost:24111"
    Source File: ../cac.cpp line 1237
    Current Time: Tue Nov 22 2022 11:45:09.509185480
..................................................................
CAC: error = "No route to host" sending UDP msg to 134.79.87.255:5064
CAC: Unable to connect because "Connection refused"
CAC: error = "Network is unreachable" sending UDP msg to 134.79.87.255:5064
CAC: ok sending UDP msg to 134.79.87.255:5064
CAC: error = "Network is unreachable" sending UDP msg to 134.79.87.255:5064
CAC: Unable to connect because "Invalid argument"
CAC: Unable to connect because "Invalid argument"
CAC: ok sending UDP msg to 134.79.87.255:5064
CAC: Unable to conne

In [90]:
run1(tao, MONITOR, DM_GOOD)
set_OTR0H04()
target_twiss_at_ENDHTR = tao.ele_twiss('ENDHTR')

dt: 0.11398601531982422


In [ ]:
# Distort the QUAD on the real machine, and run this again
run1(tao, MONITOR, DM_GOOD)

In [93]:
dat_name = "HTR.endtwiss"
var_name = "q.HTR[5:9]"
desired_twiss = target_twiss_at_ENDHTR


dict_new_quads = twiss_optimizer2('Q0H01', -4.5 ,  dat_name, var_name, desired_twiss, reset_lat=False)
dict_new_quads

Old merit: 3.66047060055099e-28
New merit: 8.61436108501345e-26


{'Q0H05': 1.99047760958474,
 'Q0H06': -0.489082889935051,
 'Q0H07': -2.08005818801471,
 'Q0H08': 2.20748978167364,
 'QHD01': 2.33337098172311}

In [83]:
def prepare_caput_values(dict_quads, df):
    """
    Given a dictionary of quad names (Bmad names) and values,
    Provide a dictionary of the corresponding quad PV names and PV values. 
    """
    
    quad_names = dict_quads.keys()
    quad_values = dict_quads.values()
    
    df2 = df[df['bmad_name'].isin(quad_names)][['bmad_name','pvname','bmad_factor']]
    print(df2)
    
    out = {}
    
    # temporary look-up dictionary (to make sure order does not matter)
    q_value = {}
    for name, value in zip(quad_names, quad_values):
        q_value[name] = value
    
    for row in df2.itertuples():
        
        key = row.pvname
        value = 1.0/row.bmad_factor * q_value[row.bmad_name]
        
        assert key not in out  # Check no repeated keys (quad PVs)
        out[key] = value
    return out

In [41]:
df = DM_GOOD['quad'].data

dict_to_caput = prepare_caput_values(dict_new_quads, df)
dict_to_caput

  bmad_name             pvname  bmad_factor
5     Q0H05  QUAD:HTR:365:BDES    -0.803859
6     Q0H06  QUAD:HTR:385:BDES    -0.803859
7     Q0H07  QUAD:HTR:440:BDES    -0.803859
8     Q0H08  QUAD:HTR:460:BDES    -0.803859
9     QHD01  QUAD:HTR:830:BDES    -0.803859


{'QUAD:HTR:365:BDES': -6.829562287617346,
 'QUAD:HTR:385:BDES': -0.022527611493915807,
 'QUAD:HTR:440:BDES': 3.884894913212843,
 'QUAD:HTR:460:BDES': -3.352605432177624,
 'QUAD:HTR:830:BDES': -2.7584432523164786}

In [53]:
original_set_values = caget_many(dict_to_caput.keys())
original_set_values

[-2.4766963, 0.610201, 2.5876414, -2.7461416, -2.9026848]

In [54]:
# To recover old setting
{list(dict_to_caput.keys())[i]:original_set_values[i] for i in range(len(original_set_values))}

{'QUAD:HTR:365:BDES': -2.4766963,
 'QUAD:HTR:385:BDES': 0.610201,
 'QUAD:HTR:440:BDES': 2.5876414,
 'QUAD:HTR:460:BDES': -2.7461416,
 'QUAD:HTR:830:BDES': -2.9026848}